Step 4

In [1]:
import pandas as pd
import os

In [2]:
import neo4j

In [3]:
from neo4j import GraphDatabase

In [4]:
from graphdatascience import GraphDataScience

In [5]:
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")

In [6]:
NEO4J_AUTH = ( "neo4j",  "june15kent")

In [7]:
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

In [ ]:
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

Step 5

In [ ]:
pii_query = '''
MATCH (c:Client)-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-> (n) <-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-(d:Client)
WHERE id(c) < id(d)
RETURN c.id, d.id, count(*) AS freq
ORDER BY freq DESC
'''

In [ ]:
gds.run_cypher(pii_query)

Step 6

In [ ]:
project_query = '''
CALL gds.graph.project(
    'clientClusters' ,
    { Client: {
        label: 'Client' }
    },
    { SHARED_PII: {
        type: 'SHARED_PII',
        orientation: 'UNDIRECTED',
        properties: {
        count: {
        property: 'count' }
            }
        }
    }
)
YIELD graphName, nodeCount, relationshipCount
'''

In [ ]:
gds.run_cypher(project_query)

Step 7

In [ ]:
streaming_query = '''
CALL gds.wcc.stream(
    'clientClusters',
    {
    nodeLabels: ['Client'],
    relationshipTypes: ['SHARED_PII'],
    consecutiveIds: true
    }
)
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).id AS clientId, 
componentId AS clusterId
'''

In [ ]:
pandasDF = driver.execute_query(
    streaming_query,
    database_="neo4j",
    result_transformer_=  neo4j.Result.to_df
)

In [ ]:
print(type(pandasDF))

In [ ]:
with driver.session() as session:
    result = session.run(streaming_query)
    print(result.data())

Step 8

In [ ]:
possible_fraud_members = '''
CALL gds.wcc.stream(
    'clientClusters',
    {
    nodeLabels: ['Client'],
    relationshipTypes: ['SHARED_PII'],
    consecutiveIds: true
    }
)
YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS clientId , componentId AS clusterId
WITH clusterId, collect(clientId.id) AS clients
WITH clusterId, clients, size(clients) AS clusterSize WHERE clusterSize >= 2
UNWIND clients AS client
MATCH (c:Client) WHERE c.id = client
SET c.secondPartyFraudRing = clusterId
'''

In [ ]:
pandasDF = driver.execute_query(
    possible_fraud_members,
    database_="neo4j",
    result_transformer_=  neo4j.Result.to_df
)

In [ ]:
with driver.session() as session:
    result = session.run(possible_fraud_members)
    print(result.single())

In [ ]:
gds.run_cypher('''CALL gds.graph.drop('clientClusters', false)''')

Step 9

In [ ]:
bipartite_graph = '''
MATCH (c:Client) WHERE c.secondPartyFraudRing is NOT NULL
WITH collect(c) as clients

MATCH (n) WHERE n:Email OR n:Phone OR n:SSN

WITH clients, collect(n) AS piis
WITH clients + piis AS nodes

MATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]->(p)
WHERE c.secondPartyFraudRing is NOT NULL

WITH nodes, collect({source: c, target: p}) as relationships

CALL gds.graph.project.cypher(
    'similarity',
    "UNWIND $nodes as n
        RETURN id(n) AS id,labels(n) AS labels",
    "UNWIND $relationships as r
        RETURN id(r['source']) AS source, id(r['target']) AS target,
    'HAS_PII' as type",
    { parameters:
        { nodes: nodes,
            relationships: relationships }
    }
)
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount

'''

In [ ]:
gds.run_cypher(bipartite_graph)

Step 10

In [38]:
nodeSimilarity_mutate = '''

CALL gds.nodeSimilarity.mutate(
  'similarity',
  {
    mutateProperty: 'jaccardScore',
    mutateRelationshipType: 'SIMILAR_TO',
    topK: 15
  }
)
YIELD
  preProcessingMillis,
  computeMillis,
  mutateMillis,
  postProcessingMillis,
  relationshipsWritten,
  nodesCompared,
  similarityDistribution,
  configuration
'''

In [39]:
gds.run_cypher(nodeSimilarity_mutate)

,preProcessingMillis,computeMillis,mutateMillis,postProcessingMillis,relationshipsWritten,nodesCompared,similarityDistribution,configuration
0,5,192,165,0,1518,336,"{'min': 0.19999980926513672, 'p5': 0.199999809...","{'mutateProperty': 'jaccardScore', 'jobId': '8..."


Step 11

In [ ]:
SIMILAR_TO_script = '''
CALL gds.graph.writeRelationship(
  'similarity', 'SIMILAR_TO', 'jaccardScore'
)
YIELD relationshipsWritten, propertiesWritten
'''

In [ ]:
gds.run_cypher(SIMILAR_TO_script)

Step 12

In [ ]:
how_many = '''
CALL gds.degree.write(
  'similarity',
  {
    nodeLabels: ['Client'],
    relationshipTypes: ['SIMILAR_TO'],
    relationshipWeightProperty: 'jaccardScore',
    writeProperty: 'secondPartyFraudScore'
  }
)
YIELD
  centralityDistribution,
  preProcessingMillis,
  computeMillis,
  postProcessingMillis,
  writeMillis,
  nodePropertiesWritten,
  configuration
  '''

In [ ]:
gds.run_cypher(how_many)

Step 13

In [ ]:
high_enough = '''
MATCH (c:Client)
WHERE c.secondPartyFraudScore IS NOT NULL
WITH percentileCont(c.secondPartyFraudScore, 0.95) AS threshold
MATCH (c:Client)
WHERE c.secondPartyFraudScore > threshold
SET c:SecondPartyFraudster
'''

In [ ]:
gds.run_cypher(high_enough)